# Prereqs

In [1]:
import pandas as pd
import gzip

# Import data and process

In [13]:
maf_path = "/Users/patrick/Documents/Data/mc3.v0.2.8.PUBLIC.maf.gz"

with gzip.open(maf_path, 'rt') as file:
    maf = pd.read_csv(file, sep = '\t', comment = '#')

print("Total mutations loaded: ", len(maf))

/var/folders/97/q2wcd1jj1kv7y34yz98f8rp00000gn/T/ipykernel_15674/3641784841.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  maf = pd.read_csv(file, sep = '\t', comment = '#')


Total mutations loaded:  3600963


In [ ]:
skcm_cases = maf[maf['Tumor_Sample_Barcode'].str.startswith('TCGA')]